In [2]:
import os
%pwd

'c:\\Users\\alexs\\OneDrive\\Desktop\\projects\\Kidney_disease_classification_Problem\\research'

In [3]:
os.chdir("../")
%pwd

'c:\\Users\\alexs\\OneDrive\\Desktop\\projects\\Kidney_disease_classification_Problem'

### entity files

In [5]:
#Entity - return type of my dataIngestion function
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

### src --> config --> update configuration Manager 

In [7]:
from Kidney_Disease_Classifier.constants import *
from Kidney_Disease_Classifier.utiles.common import read_yaml,create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

### data_ingestion Component

In [9]:
import os
import zipfile
import gdown
from Kidney_Disease_Classifier import logger
from Kidney_Disease_Classifier.utiles.common import get_size

class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config = config
    
    def download_file(self)-> str: #fetch data from URL
        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion",exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")
            
            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)
            
            logger.info(f"Dowloaded data from {dataset_url} into file {zip_download_dir}")
            
            
        except Exception as e:
            raise e
    
    def extract_zip_file(self):
        unzip_file_path = self.config.unzip_dir
        os.makedirs(unzip_file_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_file_path)
        
        
        

### pipeline

In [ ]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

  1%|          | 524k/57.7M [00:20<08:53, 107kB/s]